<a href="https://colab.research.google.com/github/mjgpinheiro/Physics_models/blob/main/CryptoPulse1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Importing necessary libraries
!pip install yfinance pandas numpy scikit-learn matplotlib ta

import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from ta.trend import MACD, SMAIndicator
from ta.momentum import StochasticOscillator, RSIIndicator
import matplotlib.pyplot as plt

# Define the stock symbols for American Express, Boeing, McDonald's, Exxon Mobil, FedEx, Netflix, Intel
stock_symbols = [
    'AXP', 'BA', 'MCD', 'XOM', 'FDX', 'NFLX', 'INTC',
    'JNJ', 'TSLA', 'V', 'PFE', 'BABA', 'CSCO', 'MSFT',
    'C', 'AMZN', 'TWTR', 'BTC-USD', 'AAPL', 'GOOGL', 'NVDA'
]
time_interval = '1h'  # 1-hour intervals

# Function to process each stock and predict buy/sell for the next hour
def predict_for_stock(stock_symbol):
    # Fetch fresh data from Yahoo Finance for the last 1 year to get more data points
    data = yf.download(stock_symbol, period='1y', interval=time_interval)

    # Feature Engineering - Moving Averages, MACD, Stochastic Oscillator, RSI

    # 1. Moving Averages (SMA - Simple Moving Averages)
    data['SMA_10'] = SMAIndicator(data['Close'], window=10).sma_indicator()
    data['SMA_30'] = SMAIndicator(data['Close'], window=30).sma_indicator()

    # 2. MACD (Moving Average Convergence Divergence)
    macd = MACD(close=data['Close'], window_slow=26, window_fast=12, window_sign=9)
    data['MACD'] = macd.macd()
    data['MACD_signal'] = macd.macd_signal()

    # 3. Stochastic Oscillator
    stoch = StochasticOscillator(high=data['High'], low=data['Low'], close=data['Close'], window=14, smooth_window=3)
    data['stoch_k'] = stoch.stoch()
    data['stoch_d'] = stoch.stoch_signal()

    # 4. Relative Strength Index (RSI)
    rsi = RSIIndicator(close=data['Close'], window=14)
    data['RSI'] = rsi.rsi()

    # Drop NaN values (as some indicators are based on rolling windows)
    data.dropna(inplace=True)

    # Define buy/sell signal: Buy if MACD > Signal and SMA_10 > SMA_30, otherwise sell
    def buy_sell_signal(row):
        if row['MACD'] > row['MACD_signal'] and row['SMA_10'] > row['SMA_30']:
            return 1  # Buy
        else:
            return 0  # Sell

    data['Signal'] = data.apply(buy_sell_signal, axis=1)

    # Check if there are enough samples after feature engineering
    if data.shape[0] > 10:  # Make sure there are enough rows

        # Define features (X) and target variable (y)
        X = data[['SMA_10', 'SMA_30', 'MACD', 'MACD_signal', 'stoch_k', 'stoch_d', 'RSI']]
        y = data['Signal']

        # Split the data into training and test sets
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, shuffle=False)

        # Hyperparameter tuning with GridSearchCV (Optional but recommended)
        param_grid = {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10],
            'min_samples_leaf': [1, 2, 4]
        }

        # Initialize RandomForest with GridSearchCV
        model = RandomForestClassifier(random_state=42)
        grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
        grid_search.fit(X_train, y_train)

        # Predict the next hour signal using the last available data point
        next_hour_data = data.iloc[-1][['SMA_10', 'SMA_30', 'MACD', 'MACD_signal', 'stoch_k', 'stoch_d', 'RSI']].values.reshape(1, -1)
        next_hour_prediction = grid_search.best_estimator_.predict(next_hour_data)[0]

        # Output decision based on next hour prediction
        if next_hour_prediction == 1:
            decision = "BUY"
        else:
            decision = "SELL"

        # Print the result for the stock
        print(f"For {stock_symbol}, the recommendation for the next hour is to {decision}.")
    else:
        print(f"Not enough data to make a prediction for {stock_symbol}.")

# Loop through each stock and make the prediction
for stock_symbol in stock_symbols:
    predict_for_stock(stock_symbol)


  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29412 sha256=4c8d4e732af36d6aed51c8394aea98c9eeb8b4e4cb20300dbfecf6f55bf0bd6f
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


[*********************100%***********************]  1 of 1 completed

Fitting 5 folds for each of 108 candidates, totalling 540 fits



/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


For AXP, the recommendation for the next hour is to SELL.


...

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


For NFLX, the recommendation for the next hour is to BUY.


[*********************100%***********************]  1 of 1 completed


Fitting 5 folds for each of 108 candidates, totalling 540 fits
For INTC, the recommendation for the next hour is to BUY.


/usr/local/lib/python3.10/dist-packages/sklearn/base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(
